In [46]:
from igloo1d import IGLOO_RETURNFULLSEQ,IGLOO
import tensorflow as tf
import h5py
from keras_generator_final import Mygenerator
from keras.models import Model, load_model
from keras.layers import multiply, UpSampling1D,Add,Masking,Flatten,Concatenate,LeakyReLU,PReLU,Input,LSTM, core, Bidirectional, CuDNNLSTM, CuDNNGRU,Reshape,Lambda,Permute,TimeDistributed,RepeatVector,ConvLSTM2D,Conv3D,Dense,UpSampling3D,Embedding, SpatialDropout1D,GRU,Add,Activation,multiply
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose,UpSampling2D,AtrousConvolution2D,Conv1D,SeparableConv2D,SeparableConv1D
from keras.layers.pooling import MaxPooling2D,MaxPooling3D,MaxPooling1D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.utils import to_categorical
from keras import backend as K
from keras.engine.topology import Layer
from keras import losses
from keras import initializers
from keras.regularizers import l2
from keras.layers.normalization import BatchNormalization
from keras.models import load_model as LM
from keras.preprocessing import text, sequence
from keras.backend import squeeze,sum
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.constraints import maxnorm
from keras.activations import tanh,softmax

input_shape=(599,2736)
 
nb_patches=200 
nb_patches_FULL=60 
nb_patches_vertical=5 
 
patch_size=4 
mDR=0.45 
MAXPOOL_size=1 
 
CONV1D_dim=10 
nb_stacks=1 
nb_stacks_full=1 
 
igloo_l2reg=0.01 
 
C1D_K=54 
 
Conv1D_dim_full_seq=30 
 
stretch_factor=13       #13 
 
add_residual=True 
add_residual_vertical=True 
build_backbone=False 
 
padding="causal" 

learning_rate = 0.00051 ##change
lstm_output_size = 300 ##change
vocab_size = 2536 ## change
lstm_layers = 3
dropout_rate = 0.22

h5f = h5py.File('embedding.h5', 'r')
balloon = h5f['embedding_dataset'][:]
h5f.close()
 
 
def get_model(): 
 
    CONC=[] 
    IGLOO_V=[] 
 
    inin = Input(shape=input_shape, name='input')
    #inin= Lambda(lambda q: q[:,1:,:]) (inin) ##theinin

    a=Conv1D(40,2,padding="causal")(inin)
    b=Conv1D(40,4,padding="causal")(inin)
    c=Conv1D(40,8,padding="causal")(inin)

    x=Concatenate(axis=-1)([a,b,c])
    x=Activation("relu")(x)
    x= BatchNormalization(axis=-1)(x)

    a=Conv1D(40,2,padding="causal")(x)
    b=Conv1D(40,4, padding="causal")(x)
    c=Conv1D(40,8, padding="causal")(x)

    x=Concatenate(axis=-1)([a,b,c])
    x=Activation("relu")(x)
    x= BatchNormalization(axis=-1)(x)

    a=Conv1D(40,2,padding="causal")(x)
    b=Conv1D(40,4,padding="causal")(x)
    c=Conv1D(40,8, padding="causal")(x)

    x=Concatenate(axis=-1)([a,b,c])
    x=Activation("relu")(x)
    x= BatchNormalization(axis=-1)(x)
    
    x=Lambda(lambda q: q[:,1:,:]) (x) ##theinin
 
    x=Conv1D(64, 1,strides=1,padding=padding)  (x) 
    x = BatchNormalization(axis=-1) (x) 
    x = Activation("relu") (x) 
    x = SpatialDropout1D(mDR) (x)

    IGLOO_V.append(IGLOO_RETURNFULLSEQ(x,nb_patches_FULL,Conv1D_dim_full_seq,patch_size=patch_size,padding_style=padding,stretch_factor=stretch_factor,l2reg=igloo_l2reg,
                                      add_residual=add_residual,nb_stacks=nb_stacks_full,build_backbone=build_backbone)) 
 

    CONC.append(IGLOO_V[0]) 
 
    for kk in range(5): 
 
        x=Conv1D(C1D_K, 1,strides=1,padding=padding)  (CONC[kk]) 
        x = BatchNormalization(axis=-1) (x) 
        x = Activation("relu") (x) 
        x = SpatialDropout1D(mDR) (x) 
 
        IGLOO_V.append(IGLOO_RETURNFULLSEQ(x,nb_patches_FULL,Conv1D_dim_full_seq,patch_size=patch_size,padding_style=padding,stretch_factor=stretch_factor,l2reg=igloo_l2reg,
                                           add_residual=add_residual,nb_stacks=nb_stacks_full,build_backbone=build_backbone)) 
 
 
        ###second residual connection 
        co=Add() ([IGLOO_V[kk+1],CONC[kk]]) 
        CONC.append(Activation("relu") (co)) 
 
 
    x=Conv1D(C1D_K, 1,strides=1,padding=padding)  (CONC[-1]) 
    x = BatchNormalization(axis=-1) (x) 
    x = Activation("relu") (x) 
    x = SpatialDropout1D(mDR) (x) 
 
    y=IGLOO(x,nb_patches,CONV1D_dim,patch_size=patch_size,return_sequences=False,l2reg=igloo_l2reg,padding_style=padding,nb_stacks=nb_stacks,DR=mDR,max_pooling_kernel=MAXPOOL_size) 
 
 
    y=Dense(64,activation='relu') (y) 
    y=Dropout(0.4) (y)
    output_1=Dense(1,activation='softmax') (y)

    word_input = Input(shape=(9,), name='decoder_input')
    

    word_embedding=Embedding(input_dim=1149, output_dim=500, name='word_embedding',trainable=False,weights=[balloon],input_length=9)
    embedded_word=word_embedding(word_input)


    input_=embedded_word
    

    #input_ = BatchNormalization(axis=-1)(input_)
    gru_out=GRU(900, activation='tanh', recurrent_activation='sigmoid', 
    dropout=0.22,return_sequences=True, return_state=False,unroll=False,reset_after=True)(input_)
    
    input_=gru_out
    
    input_ = BatchNormalization(axis=-1)(input_)
    gru_out=GRU(1000, activation='tanh', recurrent_activation='sigmoid', 
    dropout=0.22,return_sequences=True, return_state=False,unroll=False,reset_after=True)(input_)
    input_ = gru_out
    
    features=Permute((2,1))(x)
 
    part1=Dense(1000)(features)
    gru_out=Permute((2,1))(gru_out)
    
    shape= K.int_shape(features)  
    
    part2=Dense(shape[1])(gru_out)
    part2=Permute((2,1))(part2)
    part3= Add()([part1,part2])
    
    score = Activation("tanh")(part3)
    part4= Dense(1)(score)
    
    attention_weights=Lambda(lambda x: softmax(x,axis=1))(part4)
    
    context_vector=multiply([attention_weights,features])
    context_vector=Lambda(lambda x: K.sum(x,axis=1))(context_vector)
    
    context_vector_mod=Dense(600)(context_vector)
    context_vector_mod = Lambda(lambda x: K.expand_dims(x, -1))(context_vector_mod)
    context_vector_mod=Permute((2,1))(context_vector_mod)
    
    gru_out_mod=Dense(600)(gru_out)

    
    input_=Concatenate(axis=1)([context_vector_mod, gru_out_mod])
    input_=Activation("tanh")(input_)


    input_ = BatchNormalization(axis=-1)(input_)
    gru_out=GRU(700, activation='tanh', recurrent_activation='sigmoid', 
    dropout=0.22,return_sequences=True, return_state=False,unroll=False,reset_after=True)(input_)
    input_= gru_out


    gru_out=Flatten()(gru_out)
    sequence_output = TimeDistributed(Dense(units=vocab_size))(gru_out)

 
    opt = optimizers.Adam(lr=0.001, clipnorm=1.0, decay=0.001) 
    model = Model(inputs=[inin,word_input],outputs=[output_1,sequence_output]) 
 
    model.compile(loss=['binary_crossentropy','categorical_crossentropy'],optimizer=opt, metrics=['accuracy']) 
 
    model.fit_generator(Mygenerator(12),epochs=300)
    
    return model 
 
 
 
 
model=get_model() 
print(model.summary())

step for main layer... 0
step for main layer... 130
step for main layer... 260
step for main layer... 390
step for main layer... 520
generating additional patches for the top end of the return sequence...
step for top layer... 0
step for top layer... 10
step for top layer... 20
step for top layer... 30
step for top layer... 40
step for top layer... 50
step for top layer... 60
step for top layer... 70
step for top layer... 80
step for top layer... 90
step for top layer... 100
step for top layer... 110
step for top layer... 120
step for top layer... 130
step for top layer... 140
step for top layer... 150
step for top layer... 160
step for top layer... 170
step for top layer... 180
step for top layer... 190
step for top layer... 200
step for top layer... 210
step for top layer... 220
step for top layer... 230
step for top layer... 240
step for top layer... 250
step for top layer... 260
step for top layer... 270
step for top layer... 280
step for top layer... 290
step for top layer... 300


step for top layer... 380
step for top layer... 390
step for top layer... 400
step for top layer... 410
step for top layer... 420
step for top layer... 430
step for top layer... 440
step for top layer... 450
step for top layer... 460
step for top layer... 470
step for top layer... 480
step for top layer... 490
step for top layer... 500
step for top layer... 510
step for top layer... 520
step for top layer... 530
step for top layer... 540
step for top layer... 550
step for top layer... 560
step for top layer... 570
step for top layer... 580
step for top layer... 590
working with horizontal regular layer...
y.shape (?, 598, 30)
y Tensor("patchy_layer_returnfullseq_157/transpose:0", shape=(598, 30, ?), dtype=float32)
M Tensor("patchy_layer_returnfullseq_157/GatherNd:0", shape=(46, 60, 120, ?), dtype=float32)
reshaped_nonzero_values Tensor("patchy_layer_returnfullseq_157/transpose_1:0", shape=(?, 46, 60, 120), dtype=float32)
PATCH_tensor1 after Tensor("patchy_layer_returnfullseq_157/transp

step for main layer... 0
step for main layer... 130
step for main layer... 260
step for main layer... 390
step for main layer... 520
generating additional patches for the top end of the return sequence...
step for top layer... 0
step for top layer... 10
step for top layer... 20
step for top layer... 30
step for top layer... 40
step for top layer... 50
step for top layer... 60
step for top layer... 70
step for top layer... 80
step for top layer... 90
step for top layer... 100
step for top layer... 110
step for top layer... 120
step for top layer... 130
step for top layer... 140
step for top layer... 150
step for top layer... 160
step for top layer... 170
step for top layer... 180
step for top layer... 190
step for top layer... 200
step for top layer... 210
step for top layer... 220
step for top layer... 230
step for top layer... 240
step for top layer... 250
step for top layer... 260
step for top layer... 270
step for top layer... 280
step for top layer... 290
step for top layer... 300


AssertionError: 

In [ ]:
from igloo1d import IGLOO_RETURNFULLSEQ,IGLOO
import h5py
import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import multiply, UpSampling1D,Add,Masking,Flatten,Concatenate,LeakyReLU,PReLU,Input,LSTM, core, Bidirectional, CuDNNLSTM, CuDNNGRU,Reshape,Lambda,Permute,TimeDistributed,RepeatVector,ConvLSTM2D,Conv3D,Dense,UpSampling3D,Embedding, SpatialDropout1D,GRU,Add,Activation,multiply
from keras.layers.core import Dropout, Lambda
from keras.layers.convolutional import Conv2D, Conv2DTranspose,UpSampling2D,AtrousConvolution2D,Conv1D,SeparableConv2D,SeparableConv1D
from keras.layers.pooling import MaxPooling2D,MaxPooling3D,MaxPooling1D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping
from keras.callbacks import Callback
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.utils import to_categorical
from keras import backend as K
from keras.engine.topology import Layer
from keras import losses
from keras import initializers
from keras.regularizers import l2
from keras.layers.normalization import BatchNormalization
from keras.models import load_model as LM
from keras.preprocessing import text, sequence
from keras.backend import squeeze,sum
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.constraints import maxnorm
from keras.activations import tanh,softmax
from keras_generator_final import Mygenerator
from keras.models import load_model

input_shape=(599,2736)
 
nb_patches=200 
nb_patches_FULL=60 
nb_patches_vertical=5 
 
patch_size=4 
mDR=0.45 
MAXPOOL_size=1 
 
CONV1D_dim=10 
nb_stacks=1 
nb_stacks_full=1 
 
igloo_l2reg=0.01 
 
C1D_K=54 
 
Conv1D_dim_full_seq=30 
 
stretch_factor=13       #13 
 
add_residual=True 
add_residual_vertical=True 
build_backbone=False 
 
padding="causal" 

learning_rate = 0.01 ##change

vocab_size = 1149 
lstm_layers = 3
dropout_rate = 0.22

h5f = h5py.File('embedding.h5', 'r')
balloony = h5f['embedding_dataset'][:]
h5f.close()

def sparse_cross_entropy(y_true, y_pred):
    
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y_true,
                                                          logits=y_pred)

    loss_mean = tf.reduce_mean(loss)

    return loss_mean
 
 
def get_model(): 
 
    CONC=[] 
    IGLOO_V=[] 
 
    inin = Input(shape=input_shape, name='input')
    
    #inin=Lambda(lambda q: q[:,1:,:]) (inin) 

    a=Conv1D(40,2,padding="causal")(inin)
    b=Conv1D(40,4,padding="causal")(inin)
    c=Conv1D(40,8,padding="causal")(inin)

    x=Concatenate(axis=-1)([a,b,c])
    x=Activation("relu")(x)
    x= BatchNormalization(axis=-1)(x)

    a=Conv1D(40,2,padding="causal")(x)
    b=Conv1D(40,4, padding="causal")(x)
    c=Conv1D(40,8, padding="causal")(x)

    x=Concatenate(axis=-1)([a,b,c])
    x=Activation("relu")(x)
    x= BatchNormalization(axis=-1)(x)

    a=Conv1D(40,2,padding="causal")(x)
    b=Conv1D(40,4,padding="causal")(x)
    c=Conv1D(40,8, padding="causal")(x)

    x=Concatenate(axis=-1)([a,b,c])
    x=Activation("relu")(x)
    x= BatchNormalization(axis=-1)(x)
    
    inin=Lambda(lambda q: q[:,1:,:]) (x)
 
    x=Conv1D(64, 1,strides=1,padding=padding)  (x) 
    x = BatchNormalization(axis=-1) (x) 
    x = Activation("relu") (x) 
    x = SpatialDropout1D(mDR) (x)

    IGLOO_V.append(IGLOO_RETURNFULLSEQ(x,nb_patches_FULL,Conv1D_dim_full_seq,patch_size=patch_size,padding_style=padding,stretch_factor=stretch_factor,l2reg=igloo_l2reg,
                                      add_residual=add_residual,nb_stacks=nb_stacks_full,build_backbone=build_backbone)) 
 

    CONC.append(IGLOO_V[0]) 
 
    for kk in range(5): 
 
        x=Conv1D(C1D_K, 1,strides=1,padding=padding)  (CONC[kk]) 
        x = BatchNormalization(axis=-1) (x) 
        x = Activation("relu") (x) 
        x = SpatialDropout1D(mDR) (x) 
 
        IGLOO_V.append(IGLOO_RETURNFULLSEQ(x,nb_patches_FULL,Conv1D_dim_full_seq,patch_size=patch_size,padding_style=padding,stretch_factor=stretch_factor,l2reg=igloo_l2reg,
                                           add_residual=add_residual,nb_stacks=nb_stacks_full,build_backbone=build_backbone)) 
 
 
        ###second residual connection 
        co=Add() ([IGLOO_V[kk+1],CONC[kk]]) 
        CONC.append(Activation("relu") (co)) 
 
 
    x=Conv1D(C1D_K, 1,strides=1,padding=padding)  (CONC[-1]) 
    x = BatchNormalization(axis=-1) (x) 
    x = Activation("relu") (x) 
    x = SpatialDropout1D(mDR) (x) 
 
    y=IGLOO(x,nb_patches,CONV1D_dim,patch_size=patch_size,return_sequences=False,l2reg=igloo_l2reg,padding_style=padding,nb_stacks=nb_stacks,DR=mDR,max_pooling_kernel=MAXPOOL_size) 
 
 
    y=Dense(64,activation='relu') (y) 
    y=Dropout(0.4) (y)
    output_1=Dense(1,activation='softmax') (y)

    word_input = Input(shape=(29,), name='decoder_input')
    
 
    embedded_word=Embedding(input_dim=1149, output_dim=500, name='word_embedding',trainable=False,weights=[balloony],input_length=29)(word_input)
   


    input_=embedded_word
    

    #input_ = BatchNormalization(axis=-1)(input_)
    gru_out=GRU(900, activation='tanh', recurrent_activation='sigmoid', 
    dropout=0.22,return_sequences=True, return_state=False,unroll=False,reset_after=True)(input_)
    
    input_=gru_out
    
    input_ = BatchNormalization(axis=-1)(input_)
    gru_out=GRU(1000, activation='tanh', recurrent_activation='sigmoid', 
    dropout=0.22,return_sequences=True, return_state=False,unroll=False,reset_after=True)(input_)
    input_ = gru_out
    
    features=Permute((2,1))(x)
 
    part1=Dense(1000)(features)
    gru_out=Permute((2,1))(gru_out)
    
    shape= K.int_shape(features)  
    
    part2=Dense(shape[1])(gru_out)
    part2=Permute((2,1))(part2)
    part3= Add()([part1,part2])
    
    score = Activation("tanh")(part3)
    part4= Dense(1)(score)
    
    attention_weights=Lambda(lambda x: softmax(x,axis=1))(part4)
    
    context_vector=multiply([attention_weights,features])
    context_vector=Lambda(lambda x: K.sum(x,axis=1))(context_vector)
    
    context_vector_mod=Dense(600)(context_vector)
    context_vector_mod = Lambda(lambda x: K.expand_dims(x, -1))(context_vector_mod)
    context_vector_mod=Permute((2,1))(context_vector_mod)
    
    gru_out_mod=Dense(600)(gru_out)

    
    input_=Concatenate(axis=1)([context_vector_mod, gru_out_mod])
    input_=Activation("tanh")(input_)


    input_ = BatchNormalization(axis=-1)(input_)
    gru_out=GRU(700, activation='tanh', recurrent_activation='sigmoid', 
    dropout=0.22,return_sequences=True, return_state=False,unroll=False,reset_after=True)(input_)
    input_= gru_out


    
    sequence_output = TimeDistributed(Dense(units=vocab_size))(gru_out)

 
    opt = optimizers.Adam(lr=0.001, clipnorm=1.0, decay=0.001) 
    model = Model(inputs=[inin,word_input],outputs=[output_1,sequence_output]) 
    

 
    model.compile(loss=['binary_crossentropy','sparse_categorical_crossentropy'],optimizer=opt, metrics=['accuracy'],loss_weights=[100,1]) 
 
    #model.fit_generator(Mygenerator(12),epochs=300)
    
    #model.save('my_eeg_model_final.h5')
    return model
    #del model
    

 
 
model=get_model()
print(model.summary())

